## EY Data Challenge - Landsat Land Surface Temperature

This sample notebook can be used to create a Landsat Land Surface Temperature (LST) product. The notebook creates a cloud-filtered median mosaic for any time period and location and then creates the LST product. A median mosaic reflects the "median" value of pixels for all spectral bands in the time series. When scenes within a time series contain clouds, the use of a median calculation can statistically remove clouds from the final median mosaic product, assuming there are plenty of clear pixels within the time series. The baseline data is [Landsat Collection-2 Level-2](https://www.usgs.gov/landsat-missions/landsat-collection-2) data from the MS Planetary Computer catalog.

In [ ]:
# Supress Warnings 
import warnings
warnings.filterwarnings('ignore')

# Import common GIS tools
import rasterio

# Import Planetary Computer tools
import pystac_client
import planetary_computer 
from odc.stac import stac_load

### Discover and load the data for analysis

First, we define our area of interest using latitude and longitude coordinates. 

In [2]:
# Define the bounding box for the entire data region using (Latitude, Longitude)
# This is the region of New York City that contains our temperature dataset
lower_left = (40.75, -74.01)
upper_right = (40.88, -73.86)

In [3]:
# Calculate the bounds for doing an archive data search
# bounds = (min_lon, min_lat, max_lon, max_lat)
bounds = (lower_left[1], lower_left[0], upper_right[1], upper_right[0]) # (min_lon, min_lat, max_lon, max_lat)

In [4]:
# Define the time window
# We will use a period of 3 months to search for data
time_window = "2021-06-01/2021-09-01"

Using the `pystac_client` we can search the Planetary Computer's STAC endpoint for items matching our query parameters. The query searches for "low cloud" scenes with overall cloud cover <20%. We will also limit our search to Landsat-8 to avoid the Landsat-7 scan line corrector failure. The result is the number of scenes matching our search criteria that touch our area of interest. Some of these may be partial scenes or contain clouds.

In [5]:
stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

search = stac.search(
    bbox=bounds, 
    datetime=time_window,
    collections=["landsat-c2-l2"],
    query={"eo:cloud_cover": {"lt": 30},"platform": {"in": ["landsat-8"]}},
)

In [6]:
items = list(search.get_items())
print('This is the number of scenes that touch our region:', len(items))

This is the number of scenes that touch our region: 7


Next, we'll load the data into an [xarray](https://xarray.pydata.org/en/stable/) DataArray using [stackstac](https://stackstac.readthedocs.io/). We will only keep the commonly used spectral bands (Red, Green, Blue, NIR, Surface Temperature). There are also several other <b>important settings for the data</b>: We have changed the projection to epsg=4326 which is standard latitude-longitude in degrees. We have specified the spatial resolution of each pixel to be 30-meters. 

In [7]:
signed_items = [planetary_computer.sign(item).to_dict() for item in items]

In [8]:
# Define the pixel resolution for the final product
# Define the scale according to our selected crs, so we will use degrees
resolution = 30  # meters per pixel 
scale = resolution / 111320.0 # degrees per pixel for crs=4326 

### Landsat Band Summary 
The following list of bands will be loaded by the Open Data Cube (ODC) stac command:<br>
We will use two load commands to separate the RGB data from the Surface Temperature data.<br><br>
Band 2 = blue = 30m<br>
Band 3 = green = 30m<br>
Band 4 = red = 30m<br>
Band 5 = nir08 (near infrared) = 30m<br>
Band 11 = Surface Temperature = lwir11 = 100m

In [9]:
data2 = stac_load(
    items,
    bands=["lwir11", "emis", "drad", "urad", "atran", "swir16", "swir22", "coastal"],
    crs="EPSG:4326", # Latitude-Longitude
    resolution=scale, # Degrees
    chunks={"x": 2048, "y": 2048},
    dtype="uint16",
    patch_url=planetary_computer.sign,
    bbox=bounds
)

In [10]:
# View the dimensions of our XARRAY and the loaded variables
# This insures we have the right coordinates and spectral bands in our xarray
display(data2)

<xarray.Dataset> Size: 30MB
Dimensions:      (latitude: 484, longitude: 558, time: 7)
Coordinates:
  * latitude     (latitude) float64 4kB 40.88 40.88 40.88 ... 40.75 40.75 40.75
  * longitude    (longitude) float64 4kB -74.01 -74.01 -74.01 ... -73.86 -73.86
    spatial_ref  int32 4B 4326
  * time         (time) datetime64[ns] 56B 2021-06-07T15:39:15.904901 ... 202...
Data variables:
    lwir11       (time, latitude, longitude) uint16 4MB dask.array<chunksize=(1, 484, 558), meta=np.ndarray>
    emis         (time, latitude, longitude) uint16 4MB dask.array<chunksize=(1, 484, 558), meta=np.ndarray>
    drad         (time, latitude, longitude) uint16 4MB dask.array<chunksize=(1, 484, 558), meta=np.ndarray>
    urad         (time, latitude, longitude) uint16 4MB dask.array<chunksize=(1, 484, 558), meta=np.ndarray>
    atran        (time, latitude, longitude) uint16 4MB dask.array<chunksize=(1, 484, 558), meta=np.ndarray>
    swir16       (time, latitude, longitude) uint16 4MB dask.array<chunksize=(1, 484, 558), meta=np.ndarray>
    swir22       (time, latitude, longitude) uint16 4MB dask.array<chunksize=(1, 484, 558), meta=np.ndarray>
    coastal      (time, latitude, longitude) uint16 4MB dask.array<chunksize=(1, 484, 558), meta=np.ndarray>

### Scaling Datasets
Landsat Collection-2 Level-2 products require scaling before creating valid output products. <br>
Scale factors for the RGB and NIR bands differ from scale factors for the Surface Temperature band.<br>

In [11]:
# Scale Factors for the lwir11, swir16, swir22, coastal bands, actually for all the surface reflectance bands
scale2 = 0.00341802 
offset2 = 149.0 
kelvin_celsius = 273.15 # convert from Kelvin to Celsius

for index in ["lwir11"]:
    data2[index] = data2[index].astype(float) * scale2 + offset2 - kelvin_celsius

for index in ["swir16", "swir22", "coastal"]:
    data2[index] = data2[index].astype(float) * scale2 + offset2

In [12]:
# Scaling for remaining attributes
def scale_factor(index):
    if index == 'emis':
        scale = 0.0001
    elif index == 'drad':
        scale = 0.001
    elif index == 'urad':
        scale = 0.001
    elif index == 'atran':
        scale = 0.0001
    elif index == 'qa':
        scale = 0.01

    return scale

for index in ["emis", "drad", "urad", "atran"]:
    factor = scale_factor(index)
    data2[index] = data2[index].astype(float) * factor
    print(f"Conversion complete for {index} using scale factor {factor}")

Conversion complete for emis using scale factor 0.0001
Conversion complete for drad using scale factor 0.001
Conversion complete for urad using scale factor 0.001
Conversion complete for atran using scale factor 0.0001


### View RGB (real color) images from the time series
You will notice that some of the scenes have clouds and some of the scenes have missing data due to scene boundary issues. Since Landsat is a descending orbit path across the equator, the time of acquisition for the scenes below is about 11:30am local time (note the time on the image is UTC time). Also, these images are merely for quick review and are not scaled correctly to reflect the proper Lat-Lon ratios. 

In [13]:
median = data2.median(dim="time").compute()

In [14]:
filename = "../data/Landsat_median_2025-03-19_v1.tiff"

In [15]:
height = median.dims["latitude"]
width = median.dims["longitude"]

# Define the Coordinate Reference System (CRS) to be common Lat-Lon coordinates
# Define the tranformation using our bounding box so the Lat-Lon information is written to the GeoTIFF
gt = rasterio.transform.from_bounds(lower_left[1], lower_left[0], upper_right[1], upper_right[0], width, height)
median.rio.write_crs("epsg:4326", inplace=True)
median.rio.write_transform(transform=gt, inplace=True)

# Create the GeoTIFF output file using the defined parameters 
with rasterio.open(filename, 'w', driver='GTiff', width=width, height=height, 
                   crs='epsg:4326', transform=gt, count=8, compress='lzw', dtype='float64') as dst:
    dst.write(median.lwir11, 1)
    dst.write(median.emis, 2)
    dst.write(median.drad, 3) 
    dst.write(median.urad, 4)
    dst.write(median.atran, 5)
    dst.write(median.swir16, 6)
    dst.write(median.swir22, 7)
    dst.write(median.coastal, 8)
    dst.close()

### Output Products

We will pick the best the best scene (no clouds, no missing data) to build our output products. This will not require any median filtering calculations as we will just pick one date from the time series. Finally, the numbering for these scenes starts at ZERO and goes from the top-left across the rows to reach the total number of scenes in the data array shown above. 